# Coursera Data Science Capstone
## Week 4 Peer-graded Assignment

# === Toronto Neighborhood Clustering ===

### <font color='red'> Disclaimer: Dear reader, as suggested by the instructions, all three assignments are within the same notebook. They are labeled as Part 1-3. </font>

##### Hendrik Welsch, January 2021

## == Part 1 ==

### Aim: Generating data frame with postal codes of the Toronto area

#### Import necessary packages

In [1]:
import pandas as pd # importing pandas for data frame work

#### Read all possible tables from the website into a list of data frames using pandas.read_html:

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # website containing postal codes
dfs = pd.read_html(url) # scrape websites for tables, save them as dataframes to list

#### Lets check how many data frames were saved:

In [3]:
len(dfs)

3

#### Lets see how the data frames look:

In [4]:
for i in dfs:
    print(i.shape, i.head(3),"\n\n")

(180, 3)   Postal Code       Borough Neighbourhood
0         M1A  Not assigned  Not assigned
1         M2A  Not assigned  Not assigned
2         M3A    North York     Parkwoods 


(4, 18)                                                   0   \
0                                                NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NL   

                                                  1   \
0                              Canadian postal codes   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NS   

                                                  2    3    4    5    6    7   \
0                                                NaN  NaN  NaN  NaN  NaN  NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...  NaN  NaN  NaN  NaN  NaN   
2                                                 PE   NB   QC   QC   QC   ON   

    8    9    10   11   12   13   14 

#### The first data frame seems to be the postal code list, while the other two are other parts of the website that pandas took for tables. 
#### We will ignore those and focus on the first data frame. Lets save it to a new name:

In [5]:
df_pc = dfs[0] # saving df at position 0
df_pc.head(11)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Now we need to clean the data frame from rows with Boroughs not assigned:

In [6]:
df_pc_clean = df_pc[df_pc.Borough != "Not assigned"] # saving df withouth not assigned postal codes as new df
df_pc_clean.head(11)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Lets reset the index to start at 0:

In [7]:
df_pc_clean.reset_index(inplace=True, drop=True)
df_pc_clean.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### As per instructions, lets look at the shape of the df:

In [8]:
df_pc_clean.shape

(103, 3)

## == Part 2 ==

### Aim: Adding coordinates to the data frame

#### <font color='red'> Disclaimer: The geocoder approach did not work for me (always returned "None"). Thus, we will use the provided csv file to add the coordinates to our dataframe </font>

#### Lets install and import wget

In [9]:
!pip install wget
import wget

#### Next we download and save the provided csv file

In [10]:
geodata = wget.download("http://cocl.us/Geospatial_data")

100% [................................................................................] 2891 / 2891

#### Now we read the downloaded csv into a data frame:

In [11]:
df_geo = pd.read_csv(geodata)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_geo.shape

(103, 3)

#### Now we need to join both data frames. This will be done based on the Postal Code, which is why we set Postal Code as the new index. Then we use the inner join approach:

In [13]:
# set index for both df to Postal Code, then join dfs
df_pc_coord = pd.concat([df_pc_clean.set_index("Postal Code"), df_geo.set_index("Postal Code")], axis=1, join="inner")
df_pc_coord.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
df_pc_coord.shape

(103, 4)

#### Now that the join is complete, we reset the index again to a number.

In [15]:
df_pc_coord.reset_index(inplace=True)
df_pc_coord.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### This is the correct data frame.

#### <font color='blue'>Lets quickly check if the join was correct by comparing the coordinates of a postal code with one listed in the example pichture in the instructions:<font>

In [16]:
df_pc_coord.loc[df_pc_coord["Postal Code"] == "M4G"]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
23,M4G,East York,Leaside,43.70906,-79.363452


#### This looks to be correct. Perfect, now we can move on to part 3!

## == Part 3 ==

#### Importing necessary packages:

In [17]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#### Installing and importing folium for maps

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [19]:
import folium # map rendering library

#### Importing geopy for determining coordinates

In [20]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



#### We use geopy to determine coordinates of Toronto

In [21]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Using the coordinates, we generate a map of Toronto with folium. Then we add all areas based on their postal codes as points of interest
#### Please note: Here, one has several options on how to separate the neighborhoods. I chose to use the postal codes. 

In [22]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for pc, lat, lng, borough, neighborhood in zip(df_pc_coord['Postal Code'], df_pc_coord['Latitude'], df_pc_coord['Longitude'], df_pc_coord['Borough'], df_pc_coord['Neighbourhood']):
    label = '{} ({}, {})'.format(neighborhood, borough, pc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### To use the Foursquare API, I have to enter my credentials (CLIENT_ID, CLIENT_SECRET). For security reasons, the cell containing this data is hidden

In [1]:
{
    "tags": [
        "hide-input",
    ]
}
CLIENT_ID = 'ULMKGONKAZW5C2KFMPGQOK1AVGPSR3HTCUEBLKZTXCB1IRP1' # your Foursquare ID
CLIENT_SECRET = 'JFUL4Y3ISFESYFNYQK3PUZIKXLJ24ZGT1BON4RE2SK30LDK3' # your Foursquare Secret

#### Then we add the version number, a limit for search queries and a radius to search around the POIs

In [24]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500

#### The following function pulls the top 100 venues around a point of interest (postal code area) and saves it to a dataframe

In [25]:
def getNearbyVenues(pcs, names, latitudes, longitudes, radius):
    
    venues_list=[]
    for pc, name, lat, lng in zip(pcs, names, latitudes, longitudes):
        #print(pc, name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pc,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Postal Code", 'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we run the above function with the toronto data as input

In [26]:
toronto_venues = getNearbyVenues(
    pcs=df_pc_coord["Postal Code"],
    names=df_pc_coord["Neighbourhood"],
    latitudes=df_pc_coord["Latitude"],
    longitudes=df_pc_coord["Longitude"], 
    radius=RADIUS)

In [27]:
toronto_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### Lets have a look at how many unique venue categories there are in Toronto:

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


#### Using one hot encoding, we generate an data frame with all collected venue categories as column labels

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [30]:
toronto_onehot.shape

(2122, 271)

#### Now we generate a new data frame containing grouped data

In [31]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [32]:
toronto_grouped.shape

(99, 271)

#### ...note that we have only 99 Postal Codes, where it was 103 in the beginning. That is because for some coordinates, the foursquare search did not come up with any venues. This might change if  the search radius is increased (currently = 500), however, for this project we will leave it at 500
#### Now we will look at the top 5 venues per location:

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                      venue  freq
0      Fast Food Restaurant   1.0
1         Accessories Store   0.0
2             Moving Target   0.0
3  Mediterranean Restaurant   0.0
4               Men's Store   0.0


----M1C----
                        venue  freq
0                         Bar   0.5
1                Home Service   0.5
2           Accessories Store   0.0
3  Modern European Restaurant   0.0
4               Movie Theater   0.0


----M1E----
                 venue  freq
0       Medical Center  0.12
1  Rental Car Location  0.12
2   Mexican Restaurant  0.12
3    Electronics Store  0.12
4                 Bank  0.12


----M1G----
                        venue  freq
0                 Coffee Shop  0.50
1           Convenience Store  0.25
2       Korean BBQ Restaurant  0.25
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


----M1H----
                  venue  freq
0       Thai Restaurant  0.12
1   Fried Chicken Joint  0.12
2  Caribbean Restaurant  0.12
3 

         venue  freq
0         Café  0.06
1  Pizza Place  0.06
2   Restaurant  0.06
3  Coffee Shop  0.06
4         Park  0.04


----M4Y----
                  venue  freq
0           Coffee Shop  0.08
1      Sushi Restaurant  0.06
2   Japanese Restaurant  0.06
3            Restaurant  0.04
4  Fast Food Restaurant  0.04


----M5A----
         venue  freq
0  Coffee Shop  0.16
1         Park  0.07
2         Café  0.07
3       Bakery  0.07
4          Pub  0.07


----M5B----
             venue  freq
0   Clothing Store  0.09
1      Coffee Shop  0.09
2   Cosmetics Shop  0.03
3             Café  0.03
4  Bubble Tea Shop  0.03


----M5C----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2         Cocktail Bar  0.04
3  American Restaurant  0.04
4            Gastropub  0.04


----M5E----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3         Cheese Shop  0.03
4  Seafood Restaurant  0.03


----

#### Next, we will define a function to return the top venues categories per location and run it for every location. The result is a data frame.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
1,M1C,Bar,Home Service,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
2,M1E,Mexican Restaurant,Breakfast Spot,Bank,Medical Center,Electronics Store,Restaurant,Intersection,Rental Car Location,Dog Run,Discount Store
3,M1G,Coffee Shop,Convenience Store,Korean BBQ Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Hakka Restaurant,Fried Chicken Joint,Bank,Caribbean Restaurant,Thai Restaurant,Gas Station,Athletics & Sports,Bakery,Drugstore,Donut Shop


#### To cluster the neighborhoods, we generate a k means model and label the locations based on their cluster.
I arbitrarily chose k=5 for clusters.

In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 2, 3, 2, 3, 3, 3, 3])

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_pc_coord

In [38]:
# merge toronto_grouped with neighborhoods_venues_sorted to add latitude/longitude for each postal code area
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Pool,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Intersection,French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Bakery,Park,Café,Pub,Theater,Breakfast Spot,Beer Store,Chocolate Shop,French Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Clothing Store,Accessories Store,Coffee Shop,Event Space,Furniture / Home Store,Boutique,Vietnamese Restaurant,Convenience Store,Diner,Falafel Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Nightclub,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
7,M3B,North York,Don Mills,43.745906,-79.352188,3.0,Japanese Restaurant,Caribbean Restaurant,Gym,Café,Athletics & Sports,Baseball Field,Donut Shop,Distribution Center,Dog Run,Doner Restaurant
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Flea Market,Pet Store,Pharmacy,Café,Breakfast Spot,Bank
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3.0,Clothing Store,Coffee Shop,Cosmetics Shop,Hotel,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Diner,Theater


#### As mentioned earlier, there are some areas without any venues (e.g. M9A), which only contain NaN in the table.

In [39]:
toronto_merged["Cluster Labels"].unique()

array([ 1.,  3., nan,  0.,  2.,  4.])

#### Lets look at the data that we can retreive from Foursquare for postal code M9A

In [40]:
df_pc_coord.loc[df_pc_coord["Postal Code"] == "M9A"] # show data frame with coordinates

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


In [41]:
# url with coordinates for M9A
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            43.667856, 
            79.532242, 
            RADIUS, 
            LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffda02076ecee301320cdda'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.6723560045, 'lng': 79.53845140585746},
   'sw': {'lat': 43.6633559955, 'lng': 79.52603259414254}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

#### Note the warning:
'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."

#### Lets drop the rows with NaN:

In [42]:
toronto_merged.dropna(subset=["Cluster Labels"], inplace=True)
toronto_merged.shape

(99, 16)

In [43]:
toronto_merged["Cluster Labels"].unique()

array([1., 3., 0., 2., 4.])

#### ... now there are only clustered areas left in the data frame!
#### We can now use folium to add the locations to a map of Toronto, where the locations is colored based on its cluster:

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
alt_colors = ["green", "blue", "red", "purple", "orange", "black"]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, pc, bor in zip(
    toronto_merged['Latitude'], 
    toronto_merged['Longitude'], 
    toronto_merged['Neighbourhood'], 
    toronto_merged['Cluster Labels'], 
    toronto_merged['Postal Code'], 
    toronto_merged['Borough']):
    
    label = folium.Popup(str(pc)+ ': ' + str(poi) + ' (Cluster ' + str(int(cluster)) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=alt_colors[int(cluster)-1],
        fill=True,
        fill_color=alt_colors[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Based on k=5 and radius of 500 m, we can describe the cluster as follows:

#### Cluster 0 (black):
These locations are defined by high numbers of home services.

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,0.0,Bar,Home Service,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
62,Central Toronto,0.0,Home Service,Garden,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant


#### Cluster 1 (green): 
Locations in this cluster are defined by their vincinity to places for outdoor activities (Parks, Trails, Playgrounds)

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Food & Drink Shop,Park,Pool,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
21,York,1.0,Park,Women's Store,Pool,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
35,East York,1.0,Park,Convenience Store,Intersection,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
40,North York,1.0,Airport,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
49,North York,1.0,Bakery,Park,Construction & Landscaping,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
52,North York,1.0,Gym,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
61,Central Toronto,1.0,Park,Bus Line,Swim School,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
66,North York,1.0,Park,Bar,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
68,Central Toronto,1.0,Park,Sushi Restaurant,Jewelry Store,Trail,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
77,Etobicoke,1.0,Pizza Place,Park,Bus Line,Sandwich Place,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Dessert Shop,Doner Restaurant


#### Cluster 2 (blue):
Cluster 2 seems to be defined by increased numbers of Pizza places. Interestingly, one location in Scarborough is the exeption with no pizza places among the top venue categories.

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Scarborough,2.0,Coffee Shop,Convenience Store,Korean BBQ Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
32,Scarborough,2.0,Pizza Place,Playground,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
50,North York,2.0,Pizza Place,Gym,Intersection,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
63,York,2.0,Pizza Place,Breakfast Spot,Convenience Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
72,North York,2.0,Pizza Place,Grocery Store,Coffee Shop,Pharmacy,Comfort Food Restaurant,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
93,Etobicoke,2.0,Pizza Place,Pub,Gym,Sandwich Place,Coffee Shop,Skating Rink,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


#### Cluster 3 (red): 
Most locations group into cluster 3. Here, coffee shops, various restaurants, shops and bars define the area. Since locations in this cluster are spread all over Toronto, one can assume that these areas are normal living/shopping areas.

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Intersection,French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,3.0,Coffee Shop,Bakery,Park,Café,Pub,Theater,Breakfast Spot,Beer Store,Chocolate Shop,French Restaurant
3,North York,3.0,Clothing Store,Accessories Store,Coffee Shop,Event Space,Furniture / Home Store,Boutique,Vietnamese Restaurant,Convenience Store,Diner,Falafel Restaurant
4,Downtown Toronto,3.0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Nightclub,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café
6,Scarborough,3.0,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
7,North York,3.0,Japanese Restaurant,Caribbean Restaurant,Gym,Café,Athletics & Sports,Baseball Field,Donut Shop,Distribution Center,Dog Run,Doner Restaurant
8,East York,3.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Flea Market,Pet Store,Pharmacy,Café,Breakfast Spot,Bank
9,Downtown Toronto,3.0,Clothing Store,Coffee Shop,Cosmetics Shop,Hotel,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Diner,Theater
10,North York,3.0,Pizza Place,Japanese Restaurant,Bakery,Pub,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dessert Shop
13,North York,3.0,Gym,Beer Store,Coffee Shop,Restaurant,Chinese Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Supermarket,Japanese Restaurant


#### Cluster 4 (purple): 
Vincinity to baseball fields seems to be the defining factor for cluster 4.

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,4.0,Food Truck,Business Service,Baseball Field,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Discount Store
57,North York,4.0,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Field
101,Etobicoke,4.0,Pool,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


## Thank you for taking your time and reviewing my work!